# What is this ?

In this baseline, we match exactly. 

In [ ]:
import pandas as pd
import os
import json

# read data

In [ ]:
df_train = pd.read_csv("/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv")

In [ ]:
df_train.head()

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [ ]:
set_of_data = set()

In [ ]:
for data_name in df_train["cleaned_label"]:
    set_of_data.add(data_name.strip())
    set_of_data.add(data_name.strip().lower())

In [ ]:
len(set_of_data)

In [ ]:
# set_of_data

# Read test data

In [ ]:
prefix_test = "/kaggle/input/coleridgeinitiative-show-us-the-data/test/"

In [ ]:
test_path = []
for dirname, _, filenames in os.walk(prefix_test):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        test_path.append(os.path.join(dirname, filename))

Load Test data

In [ ]:
pub_list = []
pub_id = []
for p in test_path:
    with open(p, "r") as f:
        pub = json.loads(f.read())
        pub_list.append(pub)
        pub_id.append(p.split("/")[-1].split(".")[0])

In [ ]:
# pub_list[0]

format into section

In [ ]:
section_list = []
for pid, pub in zip(pub_id, pub_list):
    for sec in pub:
        section_list.append(sec)
        sec["id"] = pid

In [ ]:
import re

txt = "The rain in Spain"
x = re.search("x", txt)
y = re.search("Spain", txt)
print(x) #this will print an object
print(y)
print(y.span())
print(txt[y.span()[0]:y.span()[1]])

In [ ]:
# section_list

We create clean text for each section list 

In [ ]:
for sec in section_list:
    sec["clean_text"] = clean_text(sec["text"])

In [ ]:
import re

We create a dict of result. 

All id must be include in submission.csv, including empty line with only id.



In [ ]:
result = {}
for sec in section_list:
    if not result.get(sec["id"], None):
        result[sec["id"]] = ""

In [ ]:

for sec in section_list:
    for data_name in set_of_data:
        clean_text = sec["clean_text"]
        x = re.search(data_name, clean_text)
        if x: 
            # print(sec["id"], data_name)
            if not result.get(sec["id"], None) or result.get(sec["id"], None) == "":
                result[sec["id"]] = data_name
            else:
                result[sec["id"]] = result[sec["id"]] + "|" + data_name


                

In [ ]:
# result

In [ ]:
pidlist = []
excerptlist = []
for pid in result.keys():
    pidlist.append(pid)
    excerptlist.append(result[pid])

In [ ]:
# excerptlist

We sort excerpt by alphabet value

In [ ]:
sort_excerptlist = []
for ex in excerptlist:
    ex_str_list = ex.split("|")
    ex_str_list.sort()
    n_ex = "|".join(ex_str_list)
    sort_excerptlist.append(n_ex)

In [ ]:
# sort_excerptlist

In [ ]:
result_pd = pd.DataFrame(data={"Id":pidlist, "PredictionString": sort_excerptlist})

In [ ]:
result_pd.head()

In [ ]:
result_pd.to_csv("submission.csv", index=False)